# Камераас машин таних (Энгийн хувилбар)
Камераас машин илрүүлэх хялбаршуулсан код

In [6]:
# Шаардлагатай сангууд 
import cv2
import numpy as np
import os

# YOLO загварын файлуудын зам
base_path = os.path.dirname(os.path.abspath('__file__'))
weights_path = os.path.join(base_path, "yolov3.weights")
cfg_path = os.path.join(base_path, "yolov3.cfg")
names_path = os.path.join(base_path, "coco.names")

# Камераар машин илрүүлэх энгийн функц
def detect_cars_simple():
    # YOLO загвар ачааллах
    print("YOLO загвар ачааллаж байна...")
    net = cv2.dnn.readNet(weights_path, cfg_path)
    with open(names_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]
    print("YOLO загвар амжилттай ачааллалаа")

    # Камер нээх
    print("Камер холбож байна...")
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Камер нээж чадсангүй!")
        return
        
    print("Камер амжилттай нээгдлээ. 'q' товчийг дарж зогсоох боломжтой.")
    
    try:
        while True:
            # Нэг frame унших
            ret, frame = cap.read()
            if not ret:
                print("Frame уншиж чадсангүй")
                break

            # Зургийг боловсруулах
            height, width = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(frame, 1/255.0, (320, 320), swapRB=True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            # Илрүүлсэн машинуудыг тэмдэглэх
            cars_detected = 0
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    
                    if confidence > 0.5 and classes[class_id] == 'car':
                        cars_detected += 1
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w/2)
                        y = int(center_y - h/2)
                        
                        # Машины хүрээ зурах
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        cv2.putText(frame, f'Car {confidence:.2f}', 
                                  (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Нийт машины тоо харуулах
            cv2.putText(frame, f'Cars: {cars_detected}', (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                       
            # Дүрс харуулах
            cv2.imshow('Car Detection', frame)
            
            # q товч дарсан эсэхийг шалгах
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        print("\nПрограм зогссон!")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("Камер амжилттай хаагдлаа")

# Програм эхлүүлэх
detect_cars_simple()

YOLO загвар ачааллаж байна...
YOLO загвар амжилттай ачааллалаа
Камер холбож байна...
YOLO загвар амжилттай ачааллалаа
Камер холбож байна...
Камер амжилттай нээгдлээ. 'q' товчийг дарж зогсоох боломжтой.
Камер амжилттай нээгдлээ. 'q' товчийг дарж зогсоох боломжтой.

Програм зогссон!

Програм зогссон!
Камер амжилттай хаагдлаа
Камер амжилттай хаагдлаа


# Камерын туршилт
Эхлээд камерын холболтыг шалгая

In [ ]:
# Камерын туршилт хийх
import cv2

def test_cameras():
    """Бүх боломжит камеруудыг шалгах"""
    working_cameras = []
    
    # Камерын индексүүдийг шалгах (0-4)
    for i in range(5):
        cap = cv2.VideoCapture(i, cv2.CAP_DSHOW)  # DirectShow драйвер ашиглах
        
        if cap.isOpened():
            ret, frame = cap.read()
            if ret:
                print(f"[OK] Камер {i} ажиллаж байна")
                working_cameras.append(i)
                # Туршилтын зураг харуулах
                cv2.imshow(f'Camera {i} Test', frame)
                cv2.waitKey(1000)  # 1 секунд хүлээх
                cv2.destroyWindow(f'Camera {i} Test')
            cap.release()
        else:
            print(f"[ERROR] Камер {i} нээж чадсангүй")
            
    return working_cameras

# Камерууд шалгах
available_cameras = test_cameras()
if available_cameras:
    print(f"\nАжиллаж буй камерууд: {available_cameras}")
    print(f"Хамгийн эхний камерыг ашиглах: {available_cameras[0]}")
else:
    print("\nБоломжтой камер олдсонгүй!")

In [ ]:
# Сонгосон камераар машин таних
def detect_cars_simple(camera_id=0):
    """Сонгосон камераас машин илрүүлэх"""
    try:
        # YOLO загвар ачааллах
        print("YOLO загвар ачааллаж байна...")
        net = cv2.dnn.readNet(weights_path, cfg_path)
        with open(names_path, 'r') as f:
            classes = [line.strip() for line in f.readlines()]
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]
        print("YOLO загвар амжилттай ачааллалаа")

        # Камер нээх
        print(f"Камер {camera_id} холбож байна...")
        cap = cv2.VideoCapture(camera_id, cv2.CAP_DSHOW)  # DirectShow драйвер ашиглах
        
        # Камерын тохиргоо
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        if not cap.isOpened():
            print(f"Камер {camera_id} нээж чадсангүй!")
            return
            
        print(f"Камер {camera_id} амжилттай нээгдлээ. 'q' товчийг дарж зогсооно.")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Камераас дүрс авч чадсангүй")
                break
                
            # Зургийг боловсруулах
            height, width = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(frame, 1/255.0, (320, 320), swapRB=True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            # Илрүүлсэн машинуудыг тэмдэглэх
            cars_detected = 0
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    
                    if confidence > 0.5 and classes[class_id] == 'car':
                        cars_detected += 1
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w/2)
                        y = int(center_y - h/2)
                        
                        # Машины хүрээ зурах
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        cv2.putText(frame, f'Car {confidence:.2f}', 
                                  (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Нийт машины тоо харуулах
            cv2.putText(frame, f'Cars: {cars_detected}', (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                       
            # Дүрс харуулах
            cv2.imshow('Car Detection', frame)
            
            # q товч дарсан эсэхийг шалгах
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except KeyboardInterrupt:
        print("\nПрограм зогссон!")
    finally:
        if 'cap' in locals():
            cap.release()
        cv2.destroyAllWindows()
        print("Камер амжилттай хаагдлаа")

# Боломжтой камераар програм эхлүүлэх  
if available_cameras:
    detect_cars_simple(available_cameras[0])
else:
    print("Камер холбоогүй байна!")